In [4]:
import pandas as pd 
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sbn

pd.set_option('display.max_columns', None)

In [7]:
meta = pd.read_csv('../data/raw/siginfo_beta.txt', sep='\t')
meta = meta[lambda x: (x.pert_type.isin(['trt_cp', 'trt_sh', 'trt_xpr', 'trt_oe'])) | (x.pert_type.str.contains('ctl'))]

meta.head()

C:\Users\natha\miniconda3\envs\gcnvae\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0,3,4,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_itime,pert_time,pert_time_unit,cell_mfc_name,pert_mfc_id,nsample,cc_q75,ss_ngene,tas,pct_self_rank_q25,wt,median_recall_rank_spearman,median_recall_rank_wtcs_50,median_recall_score_spearman,median_recall_score_wtcs_50,batch_effect_tstat,batch_effect_tstat_pct,is_hiq,qc_pass,pert_id,sig_id,pert_type,cell_iname,det_wells,det_plates,distil_ids,build_name,project_code,cmap_name,is_ncs_exemplar
0,b17,NaN,100.0,ug/ml,100 ug/ml,336 h,336.0,h,N8,BRD-U44432129,4,0.6164,446,0.530187,0.000000,"0.26,0.26,0.22,0.26",0.925926,1.157410,0.548655,0.705263,-2.31,0.488085,1,1,BRD-U44432129,MET001_N8_XH:BRD-U44432129:100:336,trt_cp,NAMEC8,H05|H06|H07|H08,MET001_N8_XH_X1_B17,MET001_N8_XH_X1_B17:H05|MET001_N8_XH_X1_B17:H0...,NaN,MET,BRD-U44432129,0
1,b15,10.0,10.0,uM,10 uM,3 h,3.0,h,A549,BRD-K81418486,3,0.4900,263,0.363000,0.544959,"0.34,0.34,0.31",1.907360,0.544959,0.426195,0.777817,17.02,71.748500,1,1,BRD-K81418486,ABY001_A549_XH:BRD-K81418486:10:3,trt_cp,A549,L04|L08|L12,ABY001_A549_XH_X1_B15,ABY001_A549_XH_X1_B15:L04|ABY001_A549_XH_X1_B1...,NaN,ABY,vorinostat,0
2,b15,2.5,2.5,uM,2.5 uM,24 h,24.0,h,HT29,BRD-K70511574,2,0.4000,138,0.237575,0.000000,"0.50,0.50",0.285714,0.285714,0.396519,0.685816,23.55,85.443600,1,1,BRD-K70511574,ABY001_HT29_XH:BRD-K70511574:2.5:24,trt_cp,HT29,E18|E22,ABY001_HT29_XH_X1_B15,ABY001_HT29_XH_X1_B15:E18|ABY001_HT29_XH_X1_B1...,NaN,ABY,HMN-214,0
3,b18,10.0,10.0,uM,10 uM,3 h,3.0,h,HME1,BRD-K81418486,1,0.0000,0,0.000000,0.000000,1.00,0.000000,0.000000,0.000000,0.000000,8.13,32.299700,0,1,BRD-K81418486,LTC002_HME1_3H:BRD-K81418486:10,trt_cp,HME1,F19,LTC002_HME1_3H_X1_B18,LTC002_HME1_3H_X1_B18:F19,NaN,LTC,vorinostat,0
4,b15,10.0,10.0,uM,10 uM,3 h,3.0,h,H1975,BRD-A61304759,3,0.3678,263,0.314495,2.624310,"0.37,0.40,0.23",13.812200,25.276200,0.217361,0.507420,32.10,93.626200,0,1,BRD-A61304759,ABY001_H1975_XH:BRD-A61304759:10:3,trt_cp,H1975,P01|P05|P09,ABY001_H1975_XH_X1_B15,ABY001_H1975_XH_X1_B15:P01|ABY001_H1975_XH_X1_...,NaN,ABY,tanespimycin,0


In [8]:
meta.shape

(1133285, 35)

In [10]:
meta.groupby('pert_time').count()['cell_mfc_name'].sort_values()

pert_time
 0.5           6
 288.0         6
 240.0         6
 336.0         6
 0.0          33
 408.0        36
 192.0        81
 1.0         147
 20.0        148
 30.0        148
 2.0         194
-666.0       257
 12.0        386
 168.0       711
 72.0       3334
 4.0        5368
 48.0       6915
 3.0       12661
 144.0     14174
 120.0     26286
 6.0      144892
 96.0     326164
 24.0     591326
Name: cell_mfc_name, dtype: int64

In [13]:
meta[lambda x: x.pert_time == 24.].groupby('pert_type').count()['cell_mfc_name'].sort_values()

pert_type
ctl_vector.cns          2
ctl_untrt.cns           6
ctl_vehicle.cns        26
trt_sh                 41
ctl_vector            147
ctl_untrt             205
trt_oe                675
ctl_vehicle         31061
trt_cp             559163
Name: cell_mfc_name, dtype: int64